# 第4回講義 演習

## 目次

- 課題. 畳み込みニューラルネットワーク(Convolutional Neural Networks)の実装と学習
    1. 畳み込みとプーリング in pytorch
        - 1.1. 畳み込み: torch.nn.functional.conv2d
        - 1.2. プーリング: torch.nn.functional.max_pool2d, torch.nn.functional.avg_pool2d, etc.
    2. 各層クラスの実装
        - 2.1. 畳み込み層
        - 2.2. プーリング層
        - 2.3.平滑化層（4次元->2次元）
        - 2.4. 全結合層
    3. MNISTを用いたクラス分類
    4. 学習

## 課題. 畳み込みニューラルネットワーク(Convolutional Neural Networks)の実装と学習

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.RandomState(1234)
random_state = 42

## 0. nn.functional

前回は実装理解の為用いませんでしたが、
`nn.functional`には、一般的な演算に用いられる関数が一通り用意されています。

これを用いることで、一つ一つ実装すると手間のかかる演算も簡単に使うことができるようになります。

In [ ]:
torch.manual_seed(34)

a = torch.arange(6, dtype=torch.float32).reshape(2, 3) - 3
print(a)
print()

# ReLU関数
b = F.relu(a)
print("ReLU関数")
print(b)
print()

# シグモイド関数"F.sigmoid"も用意されているがdepricatedであり、torch.sigmoidが推奨
b = torch.sigmoid(a)
print("シグモイド関数")
print(b)
print()

# ソフトマックス関数
b = F.softmax(a, dim=-1)  # 正規化したいdimを指定する. 最後のdimに対して行いたいときはdim=-1.
print("ソフトマックス関数")
print(b)
print()


a = np.array([[1, 1, 1, 0, 0],
              [0, 1, 1, 1, 0],
              [0, 0, 1, 1, 1],
              [0, 0, 1, 1, 0],
              [0, 1, 1, 0, 0]]
             ).astype('float32').reshape(1, 1, 5, 5)
a = torch.tensor(a)

# パラメータ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(1, 1, 3, 3)
W = torch.tensor(W)

b = F.conv2d(a, W, bias=None, stride=(1, 1), padding=0)

print("畳み込み演算")
print(b)

## 1. 畳み込みとプーリング in pytorch

### 1.1. 畳み込み: torch.nn.functional.conv2d [\[link\]](https://pytorch.org/docs/stable/nn.functional.html#conv2d)

- 入力または隠れ層$X_{i, j}^{k}$
    - 次元数4$(n,k,i,j)$
        - $n$：バッチサイズ
        - $k$：入力のチャネル数
        - $i$：入力の行数
        - $j$：入力の列数
- 畳み込みのフィルタ（重み）$W_{i,j}^{k,l}$
    - 次元数4$(l,k,i,j)$
        - $l$: 出力のチャネル数(フィルタ数)
        - $k$: 入力のチャネル数
        - $i$: フィルタの行数
        - $j$: フィルタの列数
    - ストライド：フィルタを適用する位置の間隔
    - ゼロパディング：入力の周りに値0の縁を加えます
        - 入力のサイズを保つ為、フィルタの縦or横の次元が$F$のときパディング数を$(F-1)/2$とします。
- フィルタ後のサイズは、入力の縦or横の次元数$N$、フィルタの縦or横の次元数$F$、ストライドの縦or横の量$S$で決まります。
    - $ceil((N-F+1)/S)$ (ceilは整数値に切り上げ)

### 畳み込みの適用例

In [ ]:
# サンプル画像
sample_image = np.array([[1, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 1, 1, 1],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0]]
                        ).astype('float32').reshape(1, 1, 5, 5)  # バッチサイズ x チャンネル数 x 高さ x 幅
sample_image = torch.tensor(sample_image)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(1, 1, 3, 3)  # 出力チャンネル数 x 入力チャンネル数 x 高さ x 幅 　
W = torch.tensor(W)

#### 畳み込み1

- バイアス: なし
- ストライド: (1, 1)
- パディング: なし
- 出力のサイズ: ceil((5-3+1)/1)=3

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(1, 1), padding=0)

print(convoluted_image)

#### 畳み込み2

- バイアス: なし
- ストライド: (2, 2)
- パディング: なし
- 出力のサイズ: ceil((5-3+1)/2)=2

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(2, 2), padding=0)

print(convoluted_image)

#### 畳み込み3
- ストライド: (1, 1)
- パディング: 上下左右に1ずつ

パディングの大きさをxとすると
出力のサイズ: ceil((5+(2*x)-3+1)/1)=5
である。（入力サイズと出力サイズが等しくなるように調整した）

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(1, 1), padding=1)

print(convoluted_image)

### 畳み込みの演習

In [ ]:
# サンプル画像
sample_image = np.array([[1, 1, 1, 0, 0, 1, 0],
                         [0, 1, 0, 1, 0, 1, 1],
                         [1, 0, 1, 1, 1, 0, 1],
                         [0, 0, 1, 1, 0, 1, 1],
                         [1, 1, 1, 1, 0, 0, 1],
                         [0, 1, 1, 1, 1, 1, 1],
                         [0, 1, 1, 0, 0, 1, 0]]
                        ).astype('float32').reshape(1, 1, 7, 7)
sample_image = torch.tensor(sample_image)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(1, 1, 3, 3)
W = torch.tensor(W)

#### 畳み込み1

- ストライド: (1, 1)
- パディング: なし ('VALID')

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(1, 1), padding=0)

print(convoluted_image)

#### 畳み込み2

- ストライド: (2, 2)
- パディング: なし

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(2, 2), padding=0)

print(convoluted_image)

#### 畳み込み3

- ストライド: (1, 1)
- パディング: 入力サイズと等しい(paddingを計算する必要あり)

In [ ]:
convoluted_image = F.conv2d(sample_image, W, bias=None, stride=(1, 1), padding=1)

print(convoluted_image)

### 1.2. プーリング: torch.nn.functional.max_pool2d \[[link\]](https://pytorch.org/docs/stable/nn.functional.html#max-pool2d), torch.nn.functional.avg_pool2d \[[link\]](https://pytorch.org/docs/stable/nn.functional.html#avg-pool2d), etc.

- プーリングには次の種類があります。
    - Max pooling
    - Sum pooling
    - Mean pooling
    - その他Lpプーリングなど
- 畳み込みと同様、ストライドやパディングも考えることがあります。
- プーリング後のサイズは、入力の縦or横の次元数$N$、ウィンドウの縦or横の次元数$W$、ストライドの縦or横の量$S$で決まります。
    - $ceil((N-W+1)/S)$  (ceilは整数値に切り上げ)

### プーリングの適用例 

In [ ]:
sample_image = np.array([[77, 80, 82, 78, 70],
                         [83, 78, 80, 83, 82],
                         [87, 82, 81, 80, 74],
                         [87, 87, 85, 77, 66],
                         [84, 79, 77, 78, 76]]
                         ).astype("float32").reshape(1, 1, 5, 5) # バッチサイズ x 高さ x 幅 x チャンネル数s

sample_image = torch.tensor(sample_image)

#### プーリング1

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: max
- ceil((5 -2+1) / 2) = 2

In [ ]:
pooled_image = F.max_pool2d(sample_image, kernel_size=(2, 2), stride=(2, 2))

print(pooled_image)

#### プーリング2

- ウィンドウサイズ: (2, 2)
- ストライド: (1, 1)
- プーリング: max
- ceil((5 -2+1) / 1) = 4

In [ ]:
pooled_image = F.max_pool2d(sample_image, kernel_size=(2, 2), stride=(1, 1))

print(pooled_image)

#### プーリング3

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: mean
- ceil((5 -2+1) / 2) = 2

In [ ]:
pooled_image = F.avg_pool2d(sample_image, kernel_size=(2, 2), stride=(2, 2))

print(pooled_image)

### プーリングの演習

In [ ]:
sample_image = np.array([[77, 80, 82, 78, 70, 76, 75],
                         [83, 78, 78, 73, 82, 82, 85],
                         [87, 82, 81, 80, 74, 88, 70],
                         [87, 87, 85, 77, 66, 83, 87],
                         [81, 83, 77, 79, 66, 83, 87],
                         [87, 87, 83, 70, 66, 83, 87],
                         [84, 79, 77, 78, 76, 75, 80]]
                         ).astype("float32").reshape(1, 1, 7, 7)

sample_image = torch.tensor(sample_image)

#### プーリング1

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: max

In [ ]:
pooled_image = F.max_pool2d(sample_image, kernel_size=(2, 2), stride=(2, 2))

print(pooled_image)

#### プーリング2

- ウィンドウサイズ: (2, 2)
- ストライド: (1, 1)
- プーリング: max

In [ ]:
pooled_image = F.max_pool2d(sample_image, kernel_size=(2, 2), stride=(1, 1))

print(pooled_image)

#### プーリング3

- ウィンドウサイズ: (2, 2)
- ストライド: (2, 2)
- プーリング: mean

In [ ]:
pooled_image = F.avg_pool2d(sample_image, kernel_size=(2, 2), stride=(2, 2))

print(pooled_image)

## 2. 各層クラスの実装

### 2.1.  畳み込み層

活性化関数としてsigmoid関数やtanh関数のような対象な関数を使用する場合は、Xavierの初期化が使われることが多いです。以下の式で表されます。

$\displaystyle U(-\sqrt{\frac{6}{n_{\mathrm{input}} + n_{\mathrm{output}}}}, \sqrt{\frac{6}{n_{\mathrm{input}} + n_{\mathrm{output}}}})$

$U$: 一様分布、 $ n_{input}$: 入力されるユニット数、$n_{output}$: 出力されるユニット数

※ $ n_{input}$は1ニューロンが前層のいくつのニューロンとつながっているかを表します。

今回の場合、非対称なReLUを活性化関数として使うので、Heの初期化を使用しています。以下の式で表されます。

$\displaystyle U(-\sqrt{\frac{6}{n_{\mathrm{input}}}}, \sqrt{\frac{6}{n_{\mathrm{input}}}})$

$U$: 一様分布、 $ n_{input}$: 入力されるユニット数

In [ ]:
class Conv(nn.Module):
    def __init__(self, filter_shape, function=lambda x: x, stride=(1, 1), padding=0):
        super().__init__()
        # Heの初期値
        fan_in = filter_shape[1] * filter_shape[2] * filter_shape[3]
        # filter_shape: (出力チャンネル数)x(入力チャンネル数)x(縦の次元数)x(横の次元数)
        fan_out = filter_shape[0] * filter_shape[2] * filter_shape[3]

        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/fan_in),
                        np.sqrt(6/fan_in),
                        size=filter_shape
                    ).astype('float32')))

        # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.b = nn.Parameter(torch.tensor(np.zeros((filter_shape[0]), dtype='float32')))
        self.function = function
        self.stride = stride
        self.padding = padding
        
    def forward(self, x):
        u = F.conv2d(x, self.W, bias=self.b, stride=self.stride, padding=self.padding)
        return self.function(u)

### 2.2. プーリング層

In [ ]:
class Pooling(nn.Module):
    def __init__(self, ksize=(2, 2), stride=(2, 2), padding=0):
        super().__init__()
        self.ksize = ksize
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=self.ksize, stride=self.stride, padding=self.padding)
      

### 2.3. 平滑化層（4次元->2次元）

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.size()[0], -1)

### 2.4. 全結合層

In [ ]:
class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        # He Initialization
        # in_dim: 入力の次元数、out_dim: 出力の次元数              
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        -np.sqrt(6/in_dim),
                        np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))

        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function
    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)


## 3. MNISTを用いたクラス分類

ここまでで定義したクラスを用いてMNISTのクラス分類を実装する。

まずはモデルと、各種ハイパーパラメータ等を定義する


In [ ]:
conv_net = nn.Sequential(
    Conv((20, 1, 5, 5), F.relu),     # 28x28x 1 -> 24x24x20
    Pooling((2, 2)),                 # 24x24x20 -> 12x12x20
    Conv((50, 20, 5, 5), F.relu),    # 12x12x20 ->  8x 8x50
    Pooling((2, 2)),                 #  8x 8x50 ->  4x 4x50
    Flatten(),
    Dense(4*4*50, 10)
)

batch_size = 100
n_epochs = 10
lr = 0.02
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


conv_net.to(device)
optimizer = optim.SGD(conv_net.parameters(), lr=lr)

MNISTデータを読み込む

In [ ]:
# torchvisionのdatasetsを使ってMNISTのデータを取得
# ミニバッチ化や前処理などの処理を行ってくれるDataLoaderを定義

dataloader_train = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=False
)

## 4. 学習

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]

        conv_net.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = conv_net.forward(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred.to("cpu") == t).float().sum().item()
        losses_train.append(loss.tolist())

    conv_net.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = conv_net.forward(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred.to("cpu") == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))